In [1]:
import sys
sys.path.append('../src')

In [8]:
print(X_train.select_dtypes(include='object').columns)


Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'ProductId', 'TransactionStartTime'],
      dtype='object')


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load processed dataset
df = pd.read_csv("../data/processed/X_transformed_named.csv")

# Drop unwanted ID/date columns
drop_cols = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'ProductId', 'TransactionStartTime']
df = df.drop(columns=[col for col in drop_cols if col in df.columns])

# Target
y = df.pop("is_high_risk")
X = df

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "ROC_AUC": roc_auc_score(y_test, y_proba)
    }

# Logistic Regression
lr = LogisticRegression(max_iter=5000)
lr_params = {
    "C": [0.01, 0.1, 1, 10],
    "solver": ["liblinear", "lbfgs"]
}
lr_search = RandomizedSearchCV(lr, lr_params, n_iter=4, cv=3, scoring='roc_auc', random_state=42)
lr_search.fit(X_train, y_train)
lr_metrics = evaluate_model(lr_search.best_estimator_, X_test, y_test)
print("🔎 Logistic Regression:", lr_metrics)


🔎 Logistic Regression: {'Accuracy': 0.9053990487639157, 'Precision': 0.7041666666666667, 'Recall': 0.3069936421435059, 'F1': 0.4275774826059456, 'ROC_AUC': 0.911310484918994}


In [7]:
import mlflow.sklearn

with mlflow.start_run(run_name="logistic_regression_model"):
    mlflow.log_params(lr_search.best_params_)
    mlflow.log_metrics(lr_metrics)
    mlflow.sklearn.log_model(lr_search.best_estimator_, "model")
    


2025/06/30 23:38:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/30 23:39:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [8]:
rf = RandomForestClassifier()
rf_params = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}
rf_search = RandomizedSearchCV(rf, rf_params, n_iter=5, cv=3, scoring='roc_auc', random_state=42)
rf_search.fit(X_train, y_train)
rf_metrics = evaluate_model(rf_search.best_estimator_, X_test, y_test)
print("🌲 Random Forest:", rf_metrics)

# Save
with mlflow.start_run(run_name="random_forest_model"):
    mlflow.log_params(rf_search.best_params_)
    mlflow.log_metrics(rf_metrics)
    mlflow.sklearn.log_model(rf_search.best_estimator_, "model")


🌲 Random Forest: {'Accuracy': 0.9959232739246329, 'Precision': 0.978809738503156, 'Recall': 0.9859218891916439, 'F1': 0.9823529411764705, 'ROC_AUC': 0.9997708013038549}


2025/06/30 23:43:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/30 23:43:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [10]:
import joblib
joblib.dump(lr_model, "../models/logistic_model.pkl")
joblib.dump(gbm_model, "../models/gbm_model.pkl")


NameError: name 'gbm_model' is not defined